In [83]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [57]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [58]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,299.40,1247.60,-0.00,21.36,955.52,13.00,171.84,4.33,178.16,220.32,...,212.10,4977.87,2.38,1539.45,64.92,43.50,95.46,45.84,49.60,97.00
1926-08-01,306.18,1246.00,-0.00,20.25,967.52,14.12,176.04,6.50,177.04,237.96,...,227.16,5058.17,1.80,1536.81,66.00,45.51,94.95,47.12,46.65,102.04
1926-09-01,313.02,1278.00,-0.00,25.74,1024.48,16.50,174.60,9.29,169.44,231.48,...,220.92,5281.55,1.90,1521.63,65.64,50.91,97.38,48.94,46.80,108.84
1926-10-01,308.73,1288.40,-0.00,26.76,1030.72,17.88,183.48,8.83,169.84,228.36,...,208.62,5269.14,1.76,1522.95,64.80,49.38,97.98,47.44,44.45,104.64
1926-11-01,307.02,1236.00,-0.00,25.86,1041.28,17.62,174.06,9.31,161.12,228.36,...,196.80,5115.11,1.48,1485.99,61.98,43.56,93.78,44.66,43.95,95.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,41006.68,505297.52,346342.32,346278.68,229020.70,41627.04,394473.12,27397.08,677271.34,252346.56,...,75465.50,812140.55,519543.20,3359902.26,696424.79,2626741.91,1581703.20,68858.24,1204022.82,1219430.46
2023-10-01,41351.20,482695.72,322070.22,328411.27,219676.30,38554.32,350182.43,25643.28,630874.36,240484.16,...,73352.80,769734.55,505316.24,3152332.77,665101.66,2520866.93,1618384.68,61585.28,1159465.58,1184147.60
2023-11-01,38939.89,453495.00,320819.40,314148.67,210891.15,32941.16,359816.49,25072.20,626535.45,250580.48,...,70811.40,721091.80,497719.17,3174484.60,653520.80,2442166.45,1674202.86,55927.10,1082860.33,1162998.24


## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [84]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_monthly = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

# Conversion de la colonne 'Date' en format datetime avec un format spécifié
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'], format='%Y')

# Ajustement de la Date au 1er juillet
#df_monthly['Date'] = df_monthly['Date'] + pd.offsets.MonthBegin(2)
df_monthly

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,1969-01-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
1,1969-01-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
2,1969-01-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
3,1969-01-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
4,1969-01-01,0.88,0.50,0.2,0.58,0.66,0.31,0.42,0.33,0.21,...,0.36,1.32,0.49,0.41,0.24,0.60,0.54,0.27,0.74,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2023-01-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
656,2023-01-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
657,2023-01-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
658,2023-01-01,0.65,0.43,0.1,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


In [66]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_monthly = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

# Conversion de la colonne 'Date' en format datetime avec un format spécifié
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'], format='%Y')

df_monthly

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
2,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
3,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
4,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1172,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1173,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1174,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [85]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

df_mom

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-0.174167,-0.276667,-0.031667,0.210000,-0.055833,-0.501667,-1.079167,-2.055833,0.217500,-1.663333,...,0.531667,-1.380833,-1.471667,0.345833,1.013333,-0.774167,-0.055000,1.764167,-0.518333,-0.546667
1969-08-01,0.258333,-0.003333,0.035833,0.118333,0.542500,-0.162500,-0.807500,-1.251667,0.436667,-1.859167,...,0.648333,-0.938333,-0.954167,0.837500,1.261667,0.057500,-0.416667,2.268333,-0.127500,0.205000
1969-09-01,-0.415833,-0.522500,0.560000,-0.374167,0.410000,-0.912500,-2.041667,-1.716667,0.231667,-2.390833,...,0.589167,-2.028333,-1.650000,0.137500,0.483333,-0.824167,-1.970000,0.829167,-0.827500,-1.676667
1969-10-01,0.505833,0.057500,1.057500,1.015000,1.315000,-0.353333,-0.831667,-1.295000,0.698333,-1.952500,...,0.575833,-1.625833,-1.025000,0.627500,0.682500,-0.157500,-1.396667,1.035833,-0.331667,-0.795833
1969-11-01,-0.173333,-0.472500,0.231667,0.345000,1.112500,-1.507500,-1.776667,-1.692500,-0.085000,-2.583333,...,-0.170833,-2.953333,-2.521667,-0.090000,-0.257500,-1.141667,-1.864167,-0.595000,-1.652500,-1.531667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.443333,0.130833,0.800833,0.579167,1.386667,0.681667,2.873333,2.539167,1.005833,1.811667,...,1.295000,1.810000,2.147500,1.186667,1.638333,0.859167,0.887500,1.377500,1.430000,2.147500
2023-10-01,-2.074167,-1.222500,0.175000,-0.571667,0.047500,-0.840000,1.854167,1.636667,0.530000,1.255833,...,0.534167,0.715833,0.934167,1.092500,0.615000,-0.496667,0.110000,0.045833,-0.054167,1.101667
2023-11-01,-2.465833,-1.114167,-0.067500,-0.516667,-0.021667,-0.790833,2.170833,0.989167,0.159167,0.682500,...,0.526667,0.895000,1.174167,1.427500,0.755000,0.169167,0.320833,-0.030833,0.355000,1.069167


## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [87]:
# Créer un objet StandardScaler
scaler = StandardScaler()

In [86]:
# Standardiser les données dans market_cap
market_cap_standardized = scaler.fit_transform(market_cap)

# Convertir les données standardisées en DataFrame
df_market_cap_standardized = pd.DataFrame(market_cap_standardized, index=market_cap.index, columns=market_cap.columns)

# Afficher le DataFrame standardisé
df_market_cap_standardized


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,-0.536231,-0.735212,-0.446956,-0.700872,-0.632664,-0.632348,-0.570553,-0.686455,-0.747653,-0.534834,...,-0.816799,-0.556473,-0.618655,-0.548777,-0.531116,-0.611865,-0.639015,-0.524892,-0.615660,-0.555765
1926-08-01,-0.535861,-0.735222,-0.446956,-0.700882,-0.632535,-0.632283,-0.570509,-0.686396,-0.747660,-0.534607,...,-0.816030,-0.556084,-0.618660,-0.548780,-0.531109,-0.611863,-0.639016,-0.524820,-0.615671,-0.555749
1926-09-01,-0.535488,-0.735004,-0.446956,-0.700832,-0.631923,-0.632145,-0.570524,-0.686320,-0.747704,-0.534690,...,-0.816348,-0.555001,-0.618659,-0.548799,-0.531112,-0.611857,-0.639010,-0.524717,-0.615671,-0.555727
1926-10-01,-0.535722,-0.734933,-0.446956,-0.700823,-0.631856,-0.632064,-0.570433,-0.686333,-0.747702,-0.534730,...,-0.816976,-0.555061,-0.618661,-0.548798,-0.531117,-0.611858,-0.639009,-0.524802,-0.615680,-0.555740
1926-11-01,-0.535815,-0.735291,-0.446956,-0.700831,-0.631743,-0.632079,-0.570530,-0.686320,-0.747752,-0.534730,...,-0.817580,-0.555808,-0.618663,-0.548844,-0.531136,-0.611865,-0.639019,-0.524959,-0.615682,-0.555769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,1.684285,2.707190,3.751757,2.478433,1.817608,1.791356,3.493528,0.058030,3.204063,2.709569,...,3.025828,3.356458,3.798206,3.692858,4.037880,2.474247,3.262872,3.364216,3.978953,3.420012
2023-10-01,1.703078,2.552831,3.457506,2.314376,1.717214,1.612394,3.037022,0.010365,2.933277,2.556922,...,2.917948,3.150884,3.677256,3.430696,3.832361,2.349855,3.353367,2.953166,3.808913,3.304968
2023-11-01,1.571545,2.353405,3.442342,2.183418,1.622829,1.285469,3.136321,-0.005156,2.907953,2.686843,...,2.788178,2.915076,3.612670,3.458674,3.756376,2.257389,3.491073,2.633380,3.516572,3.236008


In [ ]:
# Standardiser les données dans market_cap
market_cap_standardized = scaler.fit_transform(market_cap)

# Convertir les données standardisées en DataFrame
df_market_cap_standardized = pd.DataFrame(market_cap_standardized, index=market_cap.index, columns=market_cap.columns)

# Afficher le DataFrame standardisé
df_market_cap_standardized

In [88]:
# Standardiser les données dans df_mom
mom_standardized = scaler.fit_transform(df_mom)

# Convertir les données standardisées en DataFrame
df_mom_standardized = pd.DataFrame(mom_standardized, index=df_mom.index, columns=df_mom.columns)

# Afficher le DataFrame standardisé
df_mom_standardized

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1969-07-01,-0.689567,-0.988299,-0.637683,-0.604571,-0.794286,-0.515915,-0.957432,-1.427232,-0.469193,-1.271476,...,-0.268799,-1.373947,-1.443476,-0.454795,-0.028210,-0.959620,-0.679834,0.458288,-0.827547,-0.498115
1969-08-01,-0.432832,-0.781008,-0.599273,-0.667004,-0.456205,-0.368395,-0.845416,-1.030329,-0.304049,-1.365325,...,-0.192402,-1.110734,-1.133417,-0.145353,0.112186,-0.492252,-0.904375,0.659333,-0.625786,-0.134649
1969-09-01,-0.833021,-1.174734,-0.301005,-1.002437,-0.531072,-0.694606,-1.354300,-1.259834,-0.458518,-1.620113,...,-0.231146,-1.759102,-1.550324,-0.585914,-0.327849,-0.987718,-1.868762,0.085440,-0.987149,-1.044523
1969-10-01,-0.285915,-0.734873,-0.017911,-0.056299,-0.019713,-0.451398,-0.855380,-1.051717,-0.106882,-1.410053,...,-0.239877,-1.519681,-1.175857,-0.277521,-0.215249,-0.613074,-1.512808,0.167852,-0.731184,-0.618599
1969-11-01,-0.689072,-1.136815,-0.487837,-0.512625,-0.134133,-0.953400,-1.245032,-1.247906,-0.697128,-1.712365,...,-0.728819,-2.309322,-2.072581,-0.729097,-0.746683,-1.166142,-1.803055,-0.482471,-1.413042,-0.974409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.849346,-0.679258,-0.163963,-0.353138,0.020781,-0.001227,0.672302,0.840668,0.124821,0.393834,...,0.231057,0.524065,0.724940,0.074404,0.325137,-0.041743,-0.094683,0.304098,0.178247,0.804640
2023-10-01,-1.817418,-1.705601,-0.520083,-1.136951,-0.735899,-0.663072,0.252069,0.395231,-0.233722,0.127464,...,-0.267162,-0.126781,-0.002026,0.015138,-0.253410,-0.803674,-0.577394,-0.226928,-0.587929,0.298931
2023-11-01,-2.049913,-1.623443,-0.658073,-1.099492,-0.774981,-0.641687,0.382640,0.075653,-0.513147,-0.147292,...,-0.272073,-0.020207,0.141769,0.225978,-0.174260,-0.429499,-0.446498,-0.257500,-0.376704,0.283216
